# Clipper Tutorial: Part 1

This tutorial will walk you through the process of starting Clipper, creating and querying a Clipper application, and deploying models to Clipper. In the first part of the demo, you will set up Clipper and create an application without involving any machine learning, demonstrating how a frontend developer or dev-ops engineer can set up and query Clipper without having to know anything about the machine-learning models involved.

As an example, this tutorial will walk you through creating an application that labels images as either pictures of birds or planes. You will use the CIFAR-10 dataset as the source of these images.


## Download the images

As the first step in the tutorial, download the CIFAR dataset that your Clipper application will work with. You can do this by specifying a download location, `cifar_loc`, and running the below code. This will make use of the provided [`download_cifar.py`](download_cifar.py).

>*This download can take some time. If it fails before you see the output "Finished downloading", go to the download location you specified, delete `cifar-10-python.tar.gz`, and attempt the download again.*

In [ ]:
cifar_loc = ""
%run ./download_cifar.py $cifar_loc

## Extract the images

Now, we must extract the data into a format we can load. This will make use of the provided [`extract_cifar.py`](extract_cifar.py)

This dataset has 50,000 training datapoints and 10,000 test datapoints. We don't need to use all of them to demonstrate how Clipper works. Feel free to adjust `max_train_datapoints` and `max_test_datapoints` or set them to `None` if you wish to use all the data available for training and testing. You can change these vaues and re-run this command in the future if you wish.

In [ ]:
max_train_datapoints = 10000
max_test_datapoints = 10000
%run ./extract_cifar.py $cifar_loc $max_train_datapoints $max_test_datapoints

# Load Cifar

The first step in building any application, using machine-learning or otherwise, is to understand the application requirements. Load the dataset into the notebook so you can examine it and better understand the dataset you will be working with. The `cifar_utils` library provides several utilities for working with CIFAR data – we will make use of one of them here.

In [ ]:
import cifar_utils
test_x, test_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=True))
no_norm_x, no_norm_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=False))

Take a look at the data you've loaded. The size and blurriness of these photos should give you a better understanding of the difficulty of the task you will ask of your machine learning models! If you'd like to see more images, increase the number of rows of images displayed -- the last argument to the function -- to a number greater than 2.

In [ ]:
%matplotlib inline
cifar_utils.show_example_images(no_norm_x, no_norm_y, 2)

# Start Clipper

Now you're ready to start Clipper! You will be using the `clipper_manager` client library to perform admninistrative commands.

> *Remember, Docker and Docker-Compose must be installed to before deploying Clipper. Visit https://docs.docker.com/compose/install/ for instructions on how to do so.*

Clipper uses Docker to manage application configurations and to deploy machine-learning models. Make sure your Docker daemon, local or remote, is up and running. You can check this by running `docker ps` in your command line – if your Docker daemon is not running, you will be told explicitly.

Starting clipper will have the following effect on your setup:  <img src="img/start_clipper.png" style="width: 500px;"/>

If you'd like to deploy Clipper locally, you can leave the `user` and `key` variables blank and set `host="localhost"`. Otherwise, you can deploy Clipper remotely to a machine that you have SSH access to. Set the `user` variable to your SSH username, the `key` variable to the path to your SSH key, and the `host` variable to the remote hostname or IP address.

In [ ]:
# clipper_manager must be on your path:
import sys
import os
sys.path.append(os.path.abspath('../../management/'))
import clipper_manager as cm
# Change the username if necessary
user = ""
# Set the path to the SSH key
key = ""
# Set the SSH host
host = ""
clipper = cm.Clipper(host, user, key)

clipper.start()

Congratulations! You now have a running Clipper instance that you can start to interact with. Think of your `clipper` Python object as a vehicle for that interaction. Try using it to list the applications deployed to this Clipper instance:

In [ ]:
clipper.list_apps()

# Create an application

In order to send query Clipper for predictions, you need to create an application. Each application specifies a name, a set of models it can query, the query input datatype, the selection policy, and a latency service level objective. Once you register an application with Clipper, the system will create two REST endpoints: one for requesting predictions and for providing feedback.

By associating the query interface with a specific application, Clipper allows frontend developers the flexibility to have multiple applications running in the same Clipper instance. Applications can request predictions from any model in Clipper. This allows a single Clipper instance to serve multiple machine-learning applications. It also provides a convenient mechanism for beta-testing or incremental rollout by creating experimental and stable applications for the same set of queries.

For this tutorial, you will create an application named "cifar_demo" and register two candidate models. Note that Clipper allows you to create the application before deploying the models. Clipper will be moving to a label-based model specification mechanism soon, so that in the future you won't have to explicitly enumerate all the models you want to query up front.

Registering the `cifar_demo` application with Clipper will have the following effect on your setup: <img src="img/register_app.png" style="width: 500px;"/>

> *Don't worry if this command seems to take a long time. Before starting Clipper, the Docker containers must be downloaded from Docker Hub. These containers are fairly large and may take awhile to download depending on the speed of your internet connection.*

In [ ]:
app_name = "cifar_demo"
candidate_models = [
    {"model_name": "sklearn_cifar", "model_version": 1},
    {"model_name": "tf_cifar", "model_version": 1},
]

clipper.register_application(
    app_name,
    candidate_models,
    "doubles",
    "EXP4",
    slo_micros=20000)

Now when you list the applications registered with Clipper, you should see the newly registered "cifar_demo" application show up!

In [ ]:
clipper.list_apps()

# Start serving

Now that you have registered an application, you can start querying the application for predictions. In this case,
Clipper has created two endpoints:
```
http://HOSTNAME:1337/cifar_demo/predict
http://HOSTNAME:1337/cifar_demo/update
```

You will now start querying Clipper with a simple Python frontend app that computes the average accuracy of the responses after every 100 requests and updates a plot of the results with every iteration.

This diagram shows how how the accuracy plot is receiving its test predictions: <img src="img/serve_predictions.png" style="width: 500px;"/>

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")
sns.despine()
import matplotlib as mpl
%matplotlib notebook
cifar_utils.run_serving_workload(host, app_name, test_x, test_y)

Because you haven't deployed any models yet, Clipper will essentially be randomly guessing to respond to the predictions. Because you have deployed a binary classification model, random guessing should result in 50% accuracy, which is what the plot should be showing.

*Now that you've created an application, it's time to move on to [part 2](tutorial_part_two.ipynb) of the tutorial, where you
will train and deploy some models to Clipper.*

# Cleanup

When you're done with the tutorial, you can run the `cleanup()` command to stop all the Docker containers.

In [ ]:
clipper.cleanup()